# 🔬 ER/PR/HER2 Embedding Visualization - Confusion Matrix Analysis

This notebook focuses on **confusion matrix-based visualization**, categorizing samples into four types:
- 🟢 **True Positive (TP)**: preds=1, targets=1 (Correctly predicted positive)
- 🔴 **False Positive (FP)**: preds=1, targets=0 (Incorrectly predicted positive)
- 🟡 **False Negative (FN)**: preds=0, targets=1 (Incorrectly predicted negative)
- 🔵 **True Negative (TN)**: preds=0, targets=0 (Correctly predicted negative)

## 📦 Install Library

In [1]:
!pip install umap-learn plotly pandas numpy scikit-learn nbformat -q

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import umap
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully")

✅ Libraries loaded successfully


## 📁 Import Data from `new` folder

Loading data with new metadata including `preds`, `targets`, and `Predicts` columns.

In [3]:
# Define data file paths - using NEW folder
biomarkers = {
    'ERBB2': 'HER2',
    # 'ESR1': 'ER',
    # 'PGR': 'PR'
}

data_dict = {}

# Load data for each biomarker
for marker, name in biomarkers.items():
    print(f"\nLoading {name} ({marker}) data...")

    # Load embeddings (768-dimensional vectors) - from root directory
    embedding_file = f'{marker}_validation_embeddings_stratify.tsv'
    embeddings = pd.read_csv(embedding_file, sep='\t', header=None)

    # Load metadata from NEW folder (with preds, targets, Predicts columns)
    metadata_file = f'{marker}_validation_metadata_stratify.tsv'
    metadata = pd.read_csv(metadata_file, sep='\t')

    # Extract all relevant columns
    sample_ids = metadata['slide_id'].values
    preds = metadata['preds'].values          # Predicted results (0/1)
    targets = metadata['targets'].values      # Ground truth (0/1)
    predicts = metadata['Predicts'].values    # Whether prediction is correct (True/False)

    # Create confusion matrix categories
    confusion_labels = []
    for i in range(len(preds)):
        if preds[i] == 1 and targets[i] == 1:
            confusion_labels.append('TP')  # True Positive
        elif preds[i] == 1 and targets[i] == 0:
            confusion_labels.append('FP')  # False Positive
        elif preds[i] == 0 and targets[i] == 1:
            confusion_labels.append('FN')  # False Negative
        else:  # preds[i] == 0 and targets[i] == 0
            confusion_labels.append('TN')  # True Negative

    # Store data
    data_dict[name] = {
        'embeddings': embeddings.values,
        'sample_ids': sample_ids,
        'preds': preds,
        'targets': targets,
        'predicts': predicts,
        'confusion_labels': np.array(confusion_labels),
        'correct': predicts.astype(bool),
        'n_samples': len(embeddings),
        'n_dims': embeddings.shape[1]
    }

    # Print detailed statistics
    tp_count = (confusion_labels == np.array('TP')).sum()
    fp_count = (confusion_labels == np.array('FP')).sum()
    fn_count = (confusion_labels == np.array('FN')).sum()
    tn_count = (confusion_labels == np.array('TN')).sum()

    accuracy = (tp_count + tn_count) / len(confusion_labels) * 100
    precision = tp_count / (tp_count + fp_count) if (tp_count + fp_count) > 0 else 0
    recall = tp_count / (tp_count + fn_count) if (tp_count + fn_count) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"  Sample count: {data_dict[name]['n_samples']}")
    print(f"  Dimensions: {data_dict[name]['n_dims']}")
    print(f"  🟢 TP (True Positive):  {tp_count:3d} ({tp_count/len(confusion_labels)*100:5.1f}%)")
    print(f"  🔵 TN (True Negative):  {tn_count:3d} ({tn_count/len(confusion_labels)*100:5.1f}%)")
    print(f"  🔴 FP (False Positive): {fp_count:3d} ({fp_count/len(confusion_labels)*100:5.1f}%)")
    print(f"  🟡 FN (False Negative): {fn_count:3d} ({fn_count/len(confusion_labels)*100:5.1f}%)")
    print(f"  📊 Accuracy:  {accuracy:.1f}%")
    print(f"  📊 Precision: {precision:.3f}")
    print(f"  📊 Recall:    {recall:.3f}")
    print(f"  📊 F1-Score:  {f1_score:.3f}")

print("\n✅ All data loaded successfully")


Loading HER2 (ERBB2) data...
  Sample count: 173
  Dimensions: 768
  🟢 TP (True Positive):   10 (  5.8%)
  🔵 TN (True Negative):  131 ( 75.7%)
  🔴 FP (False Positive):   6 (  3.5%)
  🟡 FN (False Negative):  26 ( 15.0%)
  📊 Accuracy:  81.5%
  📊 Precision: 0.625
  📊 Recall:    0.278
  📊 F1-Score:  0.385

✅ All data loaded successfully


## 🔄 Dimensionality Reduction

Using two methods:
- **UMAP**: Preserves local and global structure, suitable for discovering clusters
- **PCA**: Linear dimensionality reduction baseline, fast and stable

In [4]:
# Dimensionality reduction parameters
random_state = 42

print("Starting 3D dimensionality reduction...\n")

for name in biomarkers.values():
    print(f"Processing {name}...")
    embeddings = data_dict[name]['embeddings']

    # PCA dimensionality reduction to 3D
    pca = PCA(n_components=3, random_state=random_state)
    pca_result = pca.fit_transform(embeddings)
    data_dict[name]['pca'] = pca_result
    print(f"  PCA explained variance ratio: {pca.explained_variance_ratio_.sum():.3f}")
    print(f"  PC1: {pca.explained_variance_ratio_[0]:.3f}, PC2: {pca.explained_variance_ratio_[1]:.3f}, PC3: {pca.explained_variance_ratio_[2]:.3f}")

    # UMAP dimensionality reduction to 3D
    umap_reducer = umap.UMAP(
        n_components=3,  # Changed to 3D
        n_neighbors=15,
        min_dist=0.1,
        metric='euclidean',
        random_state=random_state
    )
    umap_result = umap_reducer.fit_transform(embeddings)
    data_dict[name]['umap'] = umap_result
    print(f"  UMAP 3D completed")

print("\n✅ 3D dimensionality reduction completed")

Starting 3D dimensionality reduction...

Processing HER2...
  PCA explained variance ratio: 0.391
  PC1: 0.161, PC2: 0.136, PC3: 0.093
  UMAP 3D completed

✅ 3D dimensionality reduction completed


## 📊 Visualization 1: 3D Confusion Matrix Visualization (UMAP)

Visualize all four categories with different colors and shapes in 3D space:
- 🟢 **TP**: Green circles
- 🔵 **TN**: Blue circles
- 🔴 **FP**: Red diamonds (larger, more prominent)
- 🟡 **FN**: Orange/Yellow squares (larger, more prominent)

In [5]:

# Create a single 3D subplot for PR biomarker
fig = go.Figure()

# Define colors and markers for each confusion matrix category
confusion_config = {
    'TP': {'color': '#27AE60', 'symbol': 'circle', 'size': 5, 'opacity': 0.6, 'name': '🟢 TP (True Positive)'},
    'TN': {'color': '#3498DB', 'symbol': 'circle', 'size': 5, 'opacity': 0.6, 'name': '🔵 TN (True Negative)'},
    'FP': {'color': '#E74C3C', 'symbol': 'diamond', 'size': 5, 'opacity': 0.9, 'name': '🔴 FP (False Positive)'},
    'FN': {'color': '#F39C12', 'symbol': 'square', 'size': 5, 'opacity': 0.9, 'name': '🟡 FN (False Negative)'}
}

# Order: correct predictions first (TP, TN), then errors (FP, FN)
category_order = ['TP', 'TN', 'FP', 'FN']

# Process PR biomarker
name = 'HER2'
umap_coords = data_dict[name]['umap']
confusion_labels = data_dict[name]['confusion_labels']
sample_ids = data_dict[name]['sample_ids']
preds = data_dict[name]['preds']
targets = data_dict[name]['targets']

# Plot each category in 3D
for category in category_order:
    mask = confusion_labels == category
    config = confusion_config[category]

    if mask.sum() > 0:  # Only plot if there are samples in this category
        fig.add_trace(
            go.Scatter3d(  # Changed to 3D scatter plot
                x=umap_coords[mask, 0],
                y=umap_coords[mask, 1],
                z=umap_coords[mask, 2],  # Added z-coordinate
                mode='markers',
                name=config['name'],
                marker=dict(
                    size=config['size'],
                    color=config['color'],
                    symbol=config['symbol'] if config['symbol'] in ['circle', 'square', 'diamond'] else 'circle',
                    opacity=config['opacity'],
                    line=dict(width=1.5, color='white')
                ),
                text=[f"Sample: {sample_ids[i]}<br>{name}<br>Category: {config['name']}<br>Pred: {preds[i]} | True: {targets[i]}"
                      for i in np.where(mask)[0]],
                hovertemplate='<b>%{text}</b><br>UMAP1: %{x:.2f}<br>UMAP2: %{y:.2f}<br>UMAP3: %{z:.2f}<extra></extra>',
                showlegend=True,
                legendgroup=category
            )
        )

# Update layout for 3D visualization
fig.update_layout(
    title='3D UMAP Confusion Matrix Visualization - PR (PGR)<br><sub>Circles=Correct predictions | Diamonds/Squares=Errors (FP/FN) - Interactive 3D plot</sub>',
    scene=dict(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        zaxis_title='UMAP 3',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)  # Set initial viewing angle
        ),
        aspectmode='cube'
    ),
    height=800,
    width=1000,
    showlegend=True,
    legend=dict(
        yanchor="middle",
        y=0.5,
        xanchor="left",
        x=1.02,
        orientation="v",
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="rgba(0, 0, 0, 0.2)",
        borderwidth=1
    )
)

fig.show()

print("\n✅ 3D UMAP confusion matrix visualization completed!")
print("🔄 Interactive features:")
print("   • Click and drag to rotate the 3D plot")
print("   • Use mouse wheel to zoom in/out")
print("   • Hold Shift and drag to pan")
print("   • Double-click to reset view")
print("   • Click legend items to show/hide categories")


✅ 3D UMAP confusion matrix visualization completed!
🔄 Interactive features:
   • Click and drag to rotate the 3D plot
   • Use mouse wheel to zoom in/out
   • Hold Shift and drag to pan
   • Double-click to reset view
   • Click legend items to show/hide categories


## 📊 Visualization 2: 3D PCA vs UMAP Comparison (Confusion Matrix)

Compare how different dimensionality reduction methods separate the four categories in 3D space

In [6]:
# Create 1 row x 2 columns subplot layout for 3D visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        'HER2 - PCA 3D', 'HER2 - UMAP 3D'
    ],
    horizontal_spacing=0.15,
    specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}]]  # Specify 3D scatter plots
)

# Iterate through one marker (PR)
for marker_idx, marker_name in enumerate(['HER2'], 1):
    confusion_labels = data_dict[marker_name]['confusion_labels']
    sample_ids = data_dict[marker_name]['sample_ids']
    preds = data_dict[marker_name]['preds']
    targets = data_dict[marker_name]['targets']

    # Iterate through two dimensionality reduction methods
    for method_idx, method in enumerate(['pca', 'umap'], 1):
        coords = data_dict[marker_name][method]

        # Plot each confusion matrix category
        for category in category_order:
            mask = confusion_labels == category
            config = confusion_config[category]

            if mask.sum() > 0:
                fig.add_trace(
                    go.Scatter3d(  # Changed to Scatter3d
                        x=coords[mask, 0],
                        y=coords[mask, 1],
                        z=coords[mask, 2],  # Added z-coordinate
                        mode='markers',
                        name=config['name'],
                        marker=dict(
                            size=config['size'],
                            color=config['color'],
                            symbol=config['symbol'] if config['symbol'] in ['circle', 'square', 'diamond'] else 'circle',
                            opacity=config['opacity'],
                            line=dict(width=1.5, color='white')
                        ),
                        text=[f"Sample: {sample_ids[i]}<br>{marker_name}<br>Category: {config['name']}<br>Pred: {preds[i]} | True: {targets[i]}"
                              for i in np.where(mask)[0]],
                        hovertemplate='<b>%{text}</b><br>X: %{x:.2f}<br>Y: %{y:.2f}<br>Z: %{z:.2f}<extra></extra>',
                        showlegend=(method_idx == 2),  # Only show legend once
                        legendgroup=category
                    ),
                    row=1, col=method_idx
                )

# Update layout for 3D visualization
fig.update_layout(
    title_text='3D PCA vs UMAP Comparison - Confusion Matrix Visualization<br><sub>Left: PCA 3D (linear) | Right: UMAP 3D (non-linear) - Interactive 3D plots</sub>',
    height=800,
    width=1400,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=1.15,
        orientation="v"
    )
)

# Update 3D scene properties
fig.update_scenes(
    xaxis_title='PC 1' if method == 'pca' else 'UMAP 1',
    yaxis_title='PC 2' if method == 'pca' else 'UMAP 2',
    zaxis_title='PC 3' if method == 'pca' else 'UMAP 3',
    camera=dict(
        eye=dict(x=1.5, y=1.5, z=1.5)  # Set initial camera position
    )
)

# Specific axis labels for each subplot
fig.update_layout(
    scene=dict(
        xaxis_title='PC 1',
        yaxis_title='PC 2',
        zaxis_title='PC 3',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    ),
    scene2=dict(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        zaxis_title='UMAP 3',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    )
)

fig.show()

print("\n✅ 3D visualization completed!")
print("💡 You can now:")
print("   • Rotate the 3D plots by clicking and dragging")
print("   • Zoom in/out using the mouse wheel")
print("   • Pan by holding Shift and dragging")
print("   • Reset view using the home button in the toolbar")


✅ 3D visualization completed!
💡 You can now:
   • Rotate the 3D plots by clicking and dragging
   • Zoom in/out using the mouse wheel
   • Pan by holding Shift and dragging
   • Reset view using the home button in the toolbar


## 📊 Visualization 3: Clustering Analysis with Confusion Matrix

Combine DBSCAN clustering with confusion matrix labels to see if errors cluster together

In [7]:
# Perform clustering for each biomarker
print("Performing DBSCAN clustering...\n")

for name in biomarkers.values():
    umap_coords = data_dict[name]['umap']

    # DBSCAN clustering - Optimized for more clusters (4 clusters instead of 2)
    dbscan = DBSCAN(eps=0.5, min_samples=2)  # eps=0.5, min_samples=2 gives 4 clusters
    clusters = dbscan.fit_predict(umap_coords)
    data_dict[name]['clusters'] = clusters

    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    n_noise = (clusters == -1).sum()

    print(f"{name}: Found {n_clusters} clusters, {n_noise} noise points")

    # Analyze confusion matrix distribution in each cluster
    confusion_labels = data_dict[name]['confusion_labels']

    print(f"\nCluster analysis for {name}:")
    print("-" * 60)
    for cluster_id in sorted(set(clusters)):
        if cluster_id == -1:
            cluster_name = "Noise"
        else:
            cluster_name = f"Cluster {cluster_id}"

        mask = clusters == cluster_id
        cluster_size = mask.sum()

        tp_count = ((confusion_labels == 'TP') & mask).sum()
        tn_count = ((confusion_labels == 'TN') & mask).sum()
        fp_count = ((confusion_labels == 'FP') & mask).sum()
        fn_count = ((confusion_labels == 'FN') & mask).sum()

        print(f"\n{cluster_name} (n={cluster_size}):")
        print(f"  🟢 TP: {tp_count:3d} ({tp_count/cluster_size*100:5.1f}%)")
        print(f"  🔵 TN: {tn_count:3d} ({tn_count/cluster_size*100:5.1f}%)")
        print(f"  🔴 FP: {fp_count:3d} ({fp_count/cluster_size*100:5.1f}%)")
        print(f"  🟡 FN: {fn_count:3d} ({fn_count/cluster_size*100:5.1f}%)")

        accuracy = (tp_count + tn_count) / cluster_size * 100
        print(f"  📊 Cluster Accuracy: {accuracy:.1f}%")

    print("\n" + "="*60)

print("\n✅ Clustering analysis completed")

Performing DBSCAN clustering...

HER2: Found 6 clusters, 6 noise points

Cluster analysis for HER2:
------------------------------------------------------------

Noise (n=6):
  🟢 TP:   0 (  0.0%)
  🔵 TN:   5 ( 83.3%)
  🔴 FP:   0 (  0.0%)
  🟡 FN:   1 ( 16.7%)
  📊 Cluster Accuracy: 83.3%

Cluster 0 (n=113):
  🟢 TP:   7 (  6.2%)
  🔵 TN:  89 ( 78.8%)
  🔴 FP:   4 (  3.5%)
  🟡 FN:  13 ( 11.5%)
  📊 Cluster Accuracy: 85.0%

Cluster 1 (n=24):
  🟢 TP:   2 (  8.3%)
  🔵 TN:  18 ( 75.0%)
  🔴 FP:   0 (  0.0%)
  🟡 FN:   4 ( 16.7%)
  📊 Cluster Accuracy: 83.3%

Cluster 2 (n=5):
  🟢 TP:   0 (  0.0%)
  🔵 TN:   5 (100.0%)
  🔴 FP:   0 (  0.0%)
  🟡 FN:   0 (  0.0%)
  📊 Cluster Accuracy: 100.0%

Cluster 3 (n=4):
  🟢 TP:   0 (  0.0%)
  🔵 TN:   2 ( 50.0%)
  🔴 FP:   0 (  0.0%)
  🟡 FN:   2 ( 50.0%)
  📊 Cluster Accuracy: 50.0%

Cluster 4 (n=3):
  🟢 TP:   0 (  0.0%)
  🔵 TN:   1 ( 33.3%)
  🔴 FP:   1 ( 33.3%)
  🟡 FN:   1 ( 33.3%)
  📊 Cluster Accuracy: 33.3%

Cluster 5 (n=18):
  🟢 TP:   1 (  5.6%)
  🔵 TN:  11 ( 61.1%

## 🔧 Testing Different Clustering Parameters

Test various DBSCAN parameters to find optimal cluster numbers

In [8]:
# Test different DBSCAN parameters to get more clusters
print("🔧 Testing different clustering parameters for more clusters...\n")

# Define parameter combinations to test
param_combinations = [
    {"eps": 0.4, "min_samples": 2, "description": "Very tight clusters"},
    {"eps": 0.5, "min_samples": 2, "description": "Tight clusters, small groups"},
    {"eps": 0.7, "min_samples": 2, "description": "Medium clusters, small groups"},
    {"eps": 0.5, "min_samples": 3, "description": "Tight clusters, medium groups"},
    {"eps": 0.8, "min_samples": 2, "description": "Looser clusters, small groups"},
    {"eps": 1.0, "min_samples": 3, "description": "Current setting (baseline)"}
]

for name in biomarkers.values():
    print(f"Testing parameters for {name}:")
    print("-" * 50)

    umap_coords = data_dict[name]['umap']

    for params in param_combinations:
        eps_val = params["eps"]
        min_samples_val = params["min_samples"]
        desc = params["description"]

        # Test clustering
        dbscan_test = DBSCAN(eps=eps_val, min_samples=min_samples_val)
        test_clusters = dbscan_test.fit_predict(umap_coords)

        n_clusters_test = len(set(test_clusters)) - (1 if -1 in test_clusters else 0)
        n_noise_test = (test_clusters == -1).sum()
        noise_percent = (n_noise_test / len(test_clusters)) * 100

        print(f"  eps={eps_val}, min_samples={min_samples_val} ({desc}):")
        print(f"    📊 Clusters: {n_clusters_test}, Noise: {n_noise_test} ({noise_percent:.1f}%)")

        # Show cluster sizes
        if n_clusters_test > 0:
            cluster_sizes = []
            for cluster_id in range(n_clusters_test):
                size = (test_clusters == cluster_id).sum()
                cluster_sizes.append(size)
            print(f"    📏 Cluster sizes: {cluster_sizes}")
        print()

    print("="*70)

print("\n💡 Parameter Guidelines:")
print("• Lower eps → More, tighter clusters")
print("• Lower min_samples → More, smaller clusters")
print("• Higher eps → Fewer, larger clusters")
print("• Higher min_samples → Fewer, larger clusters")
print("• Balance between too many tiny clusters vs too few large clusters")
print("\n✅ Parameter testing completed!")

🔧 Testing different clustering parameters for more clusters...

Testing parameters for HER2:
--------------------------------------------------
  eps=0.4, min_samples=2 (Very tight clusters):
    📊 Clusters: 19, Noise: 17 (9.8%)
    📏 Cluster sizes: [np.int64(7), np.int64(31), np.int64(4), np.int64(5), np.int64(23), np.int64(3), np.int64(5), np.int64(5), np.int64(7), np.int64(4), np.int64(17), np.int64(3), np.int64(11), np.int64(3), np.int64(2), np.int64(3), np.int64(18), np.int64(3), np.int64(2)]

  eps=0.5, min_samples=2 (Tight clusters, small groups):
    📊 Clusters: 6, Noise: 6 (3.5%)
    📏 Cluster sizes: [np.int64(113), np.int64(24), np.int64(5), np.int64(4), np.int64(3), np.int64(18)]

  eps=0.7, min_samples=2 (Medium clusters, small groups):
    📊 Clusters: 4, Noise: 0 (0.0%)
    📏 Cluster sizes: [np.int64(128), np.int64(25), np.int64(2), np.int64(18)]

  eps=0.5, min_samples=3 (Tight clusters, medium groups):
    📊 Clusters: 6, Noise: 6 (3.5%)
    📏 Cluster sizes: [np.int64(113

## ? Visualization 4: Cluster Center Analysis - Finding Representative Correct Samples

Find the correctly classified sample closest to each cluster center and extract its 768-dimensional embedding

In [9]:
import numpy as np
from scipy.spatial.distance import cdist

print("🔍 Finding cluster center representatives...\n")

# Analyze each biomarker
for name in biomarkers.values():
    print(f"\n{'='*70}")
    print(f"📊 {name} - Cluster Center Analysis")
    print('='*70)

    # Get data
    umap_coords = data_dict[name]['umap']
    clusters = data_dict[name]['clusters']
    confusion_labels = data_dict[name]['confusion_labels']
    sample_ids = data_dict[name]['sample_ids']
    embeddings = data_dict[name]['embeddings']  # 768-dimensional original embeddings

    # Define correct samples (TP and TN)
    correct_mask = (confusion_labels == 'TP') | (confusion_labels == 'TN')

    print(f"Total samples: {len(umap_coords)}")
    print(f"Correct samples: {correct_mask.sum()} ({correct_mask.sum()/len(umap_coords)*100:.1f}%)")

    # Store cluster center representatives
    cluster_representatives = {}

    # Analyze each cluster
    unique_clusters = sorted(set(clusters))
    for cluster_id in unique_clusters:
        if cluster_id == -1:
            cluster_name = "Noise Points"
            continue  # Skip noise points for center analysis
        else:
            cluster_name = f"Cluster {cluster_id}"

        print(f"\n🔍 {cluster_name}:")
        print("-" * 50)

        # Get all points in this cluster
        cluster_mask = clusters == cluster_id
        cluster_points = umap_coords[cluster_mask]
        cluster_sample_ids = sample_ids[cluster_mask]
        cluster_confusion = confusion_labels[cluster_mask]
        cluster_embeddings = embeddings[cluster_mask]

        # Calculate cluster center (centroid)
        cluster_center = np.mean(cluster_points, axis=0)

        print(f"Cluster size: {len(cluster_points)}")
        print(f"Cluster center (3D UMAP): [{cluster_center[0]:.3f}, {cluster_center[1]:.3f}, {cluster_center[2]:.3f}]")

        # Find correct samples within this cluster
        cluster_correct_mask = (cluster_confusion == 'TP') | (cluster_confusion == 'TN')

        if cluster_correct_mask.sum() == 0:
            print("❌ No correctly classified samples in this cluster!")
            continue

        # Get correct samples in this cluster
        correct_points_in_cluster = cluster_points[cluster_correct_mask]
        correct_sample_ids_in_cluster = cluster_sample_ids[cluster_correct_mask]
        correct_confusion_in_cluster = cluster_confusion[cluster_correct_mask]
        correct_embeddings_in_cluster = cluster_embeddings[cluster_correct_mask]

        print(f"Correct samples in cluster: {len(correct_points_in_cluster)}")

        # Calculate distances from cluster center to all correct points in cluster
        distances_to_center = cdist([cluster_center], correct_points_in_cluster, metric='euclidean')[0]

        # Find the closest correct sample to cluster center
        closest_idx = np.argmin(distances_to_center)
        closest_distance = distances_to_center[closest_idx]

        # Get information about the closest correct sample
        representative_point = correct_points_in_cluster[closest_idx]
        representative_sample_id = correct_sample_ids_in_cluster[closest_idx]
        representative_confusion = correct_confusion_in_cluster[closest_idx]
        representative_embedding = correct_embeddings_in_cluster[closest_idx]

        # Store the representative
        cluster_representatives[cluster_id] = {
            'sample_id': representative_sample_id,
            'confusion_label': representative_confusion,
            'umap_coords': representative_point,
            'distance_to_center': closest_distance,
            'embedding_768d': representative_embedding,
            'cluster_center': cluster_center,
            'cluster_size': len(cluster_points)
        }

        print(f"📍 Representative sample (closest to center):")
        print(f"   Sample ID: {representative_sample_id}")
        print(f"   Classification: {representative_confusion}")
        print(f"   3D UMAP coords: [{representative_point[0]:.3f}, {representative_point[1]:.3f}, {representative_point[2]:.3f}]")
        print(f"   Distance to center: {closest_distance:.3f}")
        print(f"   768D embedding shape: {representative_embedding.shape}")
        print(f"   First 10 embedding values: {representative_embedding[:10]}")

        # Calculate statistics for this cluster
        tp_in_cluster = (cluster_confusion == 'TP').sum()
        tn_in_cluster = (cluster_confusion == 'TN').sum()
        fp_in_cluster = (cluster_confusion == 'FP').sum()
        fn_in_cluster = (cluster_confusion == 'FN').sum()
        cluster_accuracy = (tp_in_cluster + tn_in_cluster) / len(cluster_confusion) * 100

        print(f"   Cluster composition: TP={tp_in_cluster}, TN={tn_in_cluster}, FP={fp_in_cluster}, FN={fn_in_cluster}")
        print(f"   Cluster accuracy: {cluster_accuracy:.1f}%")

    # Store representatives in data_dict for later use
    data_dict[name]['cluster_representatives'] = cluster_representatives

    print(f"\n✅ Found {len(cluster_representatives)} cluster representatives for {name}")

print(f"\n{'='*70}")
print("✅ Cluster center analysis completed!")
print(f"{'='*70}")

🔍 Finding cluster center representatives...


📊 HER2 - Cluster Center Analysis
Total samples: 173
Correct samples: 141 (81.5%)

🔍 Cluster 0:
--------------------------------------------------
Cluster size: 113
Cluster center (3D UMAP): [7.352, 1.543, 3.746]
Correct samples in cluster: 96
📍 Representative sample (closest to center):
   Sample ID: TCGA-E2-A14T-01Z-00-DX1.61B4C988-6D75-447B-A5F4-9DE92CEACC9F
   Classification: TN
   3D UMAP coords: [7.640, 1.608, 3.811]
   Distance to center: 0.302
   768D embedding shape: (768,)
   First 10 embedding values: [ 0.20800781 -0.03152466 -0.17333984  0.20922852 -0.34960938  0.13964844
  0.0748291  -0.45825195 -0.24206543 -0.01072693]
   Cluster composition: TP=7, TN=89, FP=4, FN=13
   Cluster accuracy: 85.0%

🔍 Cluster 1:
--------------------------------------------------
Cluster size: 24
Cluster center (3D UMAP): [4.780, 2.553, 1.370]
Correct samples in cluster: 20
📍 Representative sample (closest to center):
   Sample ID: TCGA-AO-A0JJ-01Z-0

In [10]:
# Create visualization showing cluster centers and their representatives
print("🎯 Creating cluster center visualization...\n")

for name in biomarkers.values():
    print(f"Creating visualization for {name}...")

    # Get data
    umap_coords = data_dict[name]['umap']
    clusters = data_dict[name]['clusters']
    confusion_labels = data_dict[name]['confusion_labels']
    sample_ids = data_dict[name]['sample_ids']

    # Create figure
    fig = go.Figure()

    # Plot all samples with confusion matrix colors (semi-transparent)
    for category in ['TP', 'TN', 'FP', 'FN']:
        mask = confusion_labels == category
        config = confusion_config[category]

        if mask.sum() > 0:
            fig.add_trace(
                go.Scatter3d(
                    x=umap_coords[mask, 0],
                    y=umap_coords[mask, 1],
                    z=umap_coords[mask, 2],
                    mode='markers',
                    name=config['name'],
                    marker=dict(
                        size=5,  # Changed from 4 to 5
                        color=config['color'],
                        opacity=0.3,  # Semi-transparent
                        line=dict(width=0.5, color='white')
                    ),
                    showlegend=True,
                    legendgroup='confusion'
                )
            )

    # Get cluster representatives
    if 'cluster_representatives' in data_dict[name]:
        cluster_representatives = data_dict[name]['cluster_representatives']

        # Plot cluster centers as large black stars
        center_coords = np.array([rep['cluster_center'] for rep in cluster_representatives.values()])
        cluster_ids = list(cluster_representatives.keys())

        if len(center_coords) > 0:
            fig.add_trace(
                go.Scatter3d(
                    x=center_coords[:, 0],
                    y=center_coords[:, 1],
                    z=center_coords[:, 2],
                    mode='markers',
                    name='🎯 Cluster Centers',
                    marker=dict(
                        size=8,  # Changed from 15 to 8 for legend/sidebar visibility
                        color='black',
                        symbol='x',
                        opacity=1.0,
                        line=dict(width=2, color='white')
                    ),
                    text=[f"Cluster {cid} Center<br>Size: {cluster_representatives[cid]['cluster_size']}"
                          for cid in cluster_ids],
                    hovertemplate='<b>%{text}</b><br>X: %{x:.3f}<br>Y: %{y:.3f}<br>Z: %{z:.3f}<extra></extra>',
                    showlegend=True,
                    legendgroup='centers'
                )
            )

            # Plot representative samples as large gold stars
            rep_coords = np.array([rep['umap_coords'] for rep in cluster_representatives.values()])
            rep_labels = [rep['confusion_label'] for rep in cluster_representatives.values()]
            rep_sample_ids = [rep['sample_id'] for rep in cluster_representatives.values()]
            rep_distances = [rep['distance_to_center'] for rep in cluster_representatives.values()]

            fig.add_trace(
                go.Scatter3d(
                    x=rep_coords[:, 0],
                    y=rep_coords[:, 1],
                    z=rep_coords[:, 2],
                    mode='markers',
                    name='⭐ Representatives',
                    marker=dict(
                        size=8,  # Changed from 12 to 8 for legend/sidebar visibility
                        color='gold',
                        symbol='diamond',  # Changed from 'star' to 'diamond'
                        opacity=1.0,
                        line=dict(width=2, color='black')
                    ),
                    text=[f"Representative<br>Cluster {cid}<br>Sample: {rep_sample_ids[i]}<br>Type: {rep_labels[i]}<br>Distance: {rep_distances[i]:.3f}"
                          for i, cid in enumerate(cluster_ids)],
                    hovertemplate='<b>%{text}</b><br>X: %{x:.3f}<br>Y: %{y:.3f}<br>Z: %{z:.3f}<extra></extra>',
                    showlegend=True,
                    legendgroup='representatives'
                )
            )

            # Draw lines connecting centers to representatives
            for i, cluster_id in enumerate(cluster_ids):
                center = cluster_representatives[cluster_id]['cluster_center']
                rep_point = cluster_representatives[cluster_id]['umap_coords']

                fig.add_trace(
                    go.Scatter3d(
                        x=[center[0], rep_point[0]],
                        y=[center[1], rep_point[1]],
                        z=[center[2], rep_point[2]],
                        mode='lines',
                        name=f'Center-Rep {cluster_id}' if i == 0 else None,
                        line=dict(color='red', width=3, dash='dash'),
                        showlegend=(i == 0),
                        legendgroup='connections',
                        hoverinfo='skip'
                    )
                )

    # Update layout
    fig.update_layout(
        title=f'{name} - Cluster Centers and Representative Samples<br><sub>🎯=Centers, ⭐=Representatives (closest correct samples), Red lines=connections</sub>',
        scene=dict(
            xaxis_title='UMAP 1',
            yaxis_title='UMAP 2',
            zaxis_title='UMAP 3',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
            aspectmode='cube'
        ),
        height=800,
        width=1200,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=1.15,
            orientation="v",
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="rgba(0, 0, 0, 0.2)",
            borderwidth=1
        )
    )

    fig.show()

print("\n✅ Cluster center visualization completed!")

🎯 Creating cluster center visualization...

Creating visualization for HER2...



✅ Cluster center visualization completed!


In [11]:
# Export representative embeddings to files for further analysis
print("💾 Exporting representative embeddings...\n")

for name in biomarkers.values():
    if 'cluster_representatives' in data_dict[name]:
        cluster_representatives = data_dict[name]['cluster_representatives']

        print(f"📊 {name} Representative Embeddings:")
        print("-" * 50)

        # Create a structured export
        export_data = []

        for cluster_id, rep_info in cluster_representatives.items():
            export_data.append({
                'cluster_id': int(cluster_id),  # Convert to regular int
                'sample_id': rep_info['sample_id'],
                'confusion_label': rep_info['confusion_label'],
                'umap_x': float(rep_info['umap_coords'][0]),  # Convert to regular float
                'umap_y': float(rep_info['umap_coords'][1]),
                'umap_z': float(rep_info['umap_coords'][2]),
                'distance_to_center': float(rep_info['distance_to_center']),
                'cluster_size': int(rep_info['cluster_size']),  # Convert to regular int
                'embedding_768d': [float(x) for x in rep_info['embedding_768d']]  # Convert all to float
            })

            print(f"Cluster {cluster_id}:")
            print(f"  📝 Sample ID: {rep_info['sample_id']}")
            print(f"  🏷️  Label: {rep_info['confusion_label']}")
            print(f"  📍 3D coords: [{rep_info['umap_coords'][0]:.3f}, {rep_info['umap_coords'][1]:.3f}, {rep_info['umap_coords'][2]:.3f}]")
            print(f"  📏 Distance: {rep_info['distance_to_center']:.3f}")
            print(f"  👥 Cluster size: {rep_info['cluster_size']}")
            print(f"  🧬 768D embedding: shape {rep_info['embedding_768d'].shape}")
            print(f"      First 5 values: {rep_info['embedding_768d'][:5]}")
            print(f"      Last 5 values: {rep_info['embedding_768d'][-5:]}")
            print(f"      Min: {rep_info['embedding_768d'].min():.4f}, Max: {rep_info['embedding_768d'].max():.4f}")
            print(f"      Mean: {rep_info['embedding_768d'].mean():.4f}, Std: {rep_info['embedding_768d'].std():.4f}")
            print()

        # Save to file
        import json
        output_file = f'cluster_representatives_{name}.json'
        with open(output_file, 'w') as f:
            json.dump(export_data, f, indent=2)

        print(f"✅ Exported {len(export_data)} representative embeddings to '{output_file}'")

        # Also save just the 768D embeddings as numpy array
        embeddings_only = np.array([rep_info['embedding_768d'] for rep_info in cluster_representatives.values()])
        np.save(f'cluster_representatives_embeddings_{name}.npy', embeddings_only)

        print(f"✅ Saved embeddings array to 'cluster_representatives_embeddings_{name}.npy'")
        print(f"   Shape: {embeddings_only.shape}")
        print()

print("🎯 Summary of cluster representatives:")
print("="*60)

for name in biomarkers.values():
    if 'cluster_representatives' in data_dict[name]:
        cluster_representatives = data_dict[name]['cluster_representatives']

        print(f"\n{name}:")
        for cluster_id, rep_info in cluster_representatives.items():
            print(f"  Cluster {cluster_id}: Sample {rep_info['sample_id']} ({rep_info['confusion_label']}) - Distance: {rep_info['distance_to_center']:.3f}")

print(f"\n{'='*60}")
print("✅ Representative embedding export completed!")
print("📁 Files created:")
print("   • cluster_representatives_PR.json (detailed info)")
print("   • cluster_representatives_embeddings_PR.npy (768D arrays only)")
print("💡 These files contain the 768-dimensional embeddings of samples")
print("   that are closest to each cluster center and correctly classified.")

💾 Exporting representative embeddings...

📊 HER2 Representative Embeddings:
--------------------------------------------------
Cluster 0:
  📝 Sample ID: TCGA-E2-A14T-01Z-00-DX1.61B4C988-6D75-447B-A5F4-9DE92CEACC9F
  🏷️  Label: TN
  📍 3D coords: [7.640, 1.608, 3.811]
  📏 Distance: 0.302
  👥 Cluster size: 113
  🧬 768D embedding: shape (768,)
      First 5 values: [ 0.20800781 -0.03152466 -0.17333984  0.20922852 -0.34960938]
      Last 5 values: [-0.61572266 -0.16992188 -0.16796875  0.8442383   0.08947754]
      Min: -1.5225, Max: 1.4697
      Mean: 0.0158, Std: 0.5157

Cluster 1:
  📝 Sample ID: TCGA-AO-A0JJ-01Z-00-DX1.D5B636F5-1B47-4033-9938-9DC8CD48CEE9
  🏷️  Label: TN
  📍 3D coords: [4.705, 2.569, 1.464]
  📏 Distance: 0.121
  👥 Cluster size: 24
  🧬 768D embedding: shape (768,)
      First 5 values: [ 1.0654297  -0.8540039  -0.32641602  0.00801086 -0.67822266]
      Last 5 values: [-1.3457031   0.11517334  0.61572266  0.35107422  0.2915039 ]
      Min: -1.7139, Max: 1.5479
      Mean: 0

## 📊 Create Enhanced PR Dataset with Cluster Information

Add cluster assignments and corresponding center embeddings to the original PR data

In [12]:
# Create enhanced dataset with cluster information for PR
print("🔄 Creating enhanced PR dataset with cluster information...\n")

# Get PR data
name = 'HER2' #PR
sample_ids = data_dict[name]['sample_ids']
preds = data_dict[name]['preds']
targets = data_dict[name]['targets']
predicts = data_dict[name]['predicts']
confusion_labels = data_dict[name]['confusion_labels']
original_embeddings = data_dict[name]['embeddings']  # 768-dim original embeddings
umap_coords = data_dict[name]['umap']
clusters = data_dict[name]['clusters']

print(f"📋 Processing {len(sample_ids)} PR samples...")

# Calculate cluster centers for each cluster
cluster_centers = {}
cluster_center_embeddings = {}

# For each cluster, calculate center in both UMAP space and find representative embedding
unique_clusters = sorted(set(clusters))
print(f"📊 Found {len(unique_clusters)} unique clusters (including noise): {unique_clusters}")

for cluster_id in unique_clusters:
    if cluster_id == -1:
        # Handle noise points - assign them to nearest cluster center
        continue

    # Get all points in this cluster
    cluster_mask = clusters == cluster_id
    cluster_umap_coords = umap_coords[cluster_mask]
    cluster_embeddings = original_embeddings[cluster_mask]

    # Calculate cluster center in UMAP space
    cluster_center_umap = np.mean(cluster_umap_coords, axis=0)
    cluster_centers[cluster_id] = cluster_center_umap

    # Find the point closest to cluster center (representative point)
    distances = np.linalg.norm(cluster_umap_coords - cluster_center_umap, axis=1)
    closest_idx = np.argmin(distances)
    representative_embedding = cluster_embeddings[closest_idx]
    cluster_center_embeddings[cluster_id] = representative_embedding

    print(f"  Cluster {cluster_id}: {cluster_mask.sum()} points, center: [{cluster_center_umap[0]:.3f}, {cluster_center_umap[1]:.3f}, {cluster_center_umap[2]:.3f}]")

# Handle noise points (-1) - assign to nearest cluster
noise_mask = clusters == -1
if noise_mask.sum() > 0:
    print(f"\n🔧 Handling {noise_mask.sum()} noise points...")
    noise_coords = umap_coords[noise_mask]

    # For each noise point, find nearest cluster center
    for i, noise_coord in enumerate(noise_coords):
        min_distance = float('inf')
        nearest_cluster = 0

        for cluster_id, center in cluster_centers.items():
            distance = np.linalg.norm(noise_coord - center)
            if distance < min_distance:
                min_distance = distance
                nearest_cluster = cluster_id

        # Assign noise point to nearest cluster for embedding purposes
        noise_indices = np.where(noise_mask)[0]
        print(f"  Noise point {i+1} assigned to nearest cluster {nearest_cluster} (distance: {min_distance:.3f})")

# Create enhanced dataset
enhanced_data = []

print(f"\n📝 Creating enhanced dataset...")

for i in range(len(sample_ids)):
    cluster_id = clusters[i]

    # Handle noise points
    if cluster_id == -1:
        # Find nearest cluster center for this noise point
        noise_coord = umap_coords[i]
        min_distance = float('inf')
        nearest_cluster = 0

        for cid, center in cluster_centers.items():
            distance = np.linalg.norm(noise_coord - center)
            if distance < min_distance:
                min_distance = distance
                nearest_cluster = cid

        # Use nearest cluster's center embedding
        center_embedding = cluster_center_embeddings[nearest_cluster]
        assigned_cluster = f"Noise->C{nearest_cluster}"
    else:
        center_embedding = cluster_center_embeddings[cluster_id]
        assigned_cluster = cluster_id

    # Create row with all information
    row_data = {
        'sample_id': sample_ids[i],
        'pred': preds[i],
        'target': targets[i],
        'predicts_correct': predicts[i],
        'confusion_label': confusion_labels[i],
        'umap_x': umap_coords[i, 0],
        'umap_y': umap_coords[i, 1],
        'umap_z': umap_coords[i, 2],
        'cluster_id': assigned_cluster,  # New column 1: cluster assignment
        'cluster_center_embedding': center_embedding.tolist()  # New column 2: 768-dim center embedding
    }

    enhanced_data.append(row_data)

print(f"✅ Enhanced dataset created with {len(enhanced_data)} rows")

# Convert to DataFrame for easier viewing and manipulation
enhanced_df = pd.DataFrame(enhanced_data)

print(f"\n📊 Enhanced Dataset Summary:")
print(f"  Shape: {enhanced_df.shape}")
print(f"  Columns: {list(enhanced_df.columns)}")
print(f"  Cluster distribution:")
cluster_counts = enhanced_df['cluster_id'].value_counts()
for cluster, count in cluster_counts.items():
    print(f"    {cluster}: {count} samples")

# Display sample rows
print(f"\n🔍 Sample of enhanced dataset:")
sample_cols = ['sample_id', 'confusion_label', 'cluster_id', 'umap_x', 'umap_y', 'umap_z']
print(enhanced_df[sample_cols].head(10).to_string(index=False))

print(f"\n💾 Cluster center embeddings info:")
for cluster_id, embedding in cluster_center_embeddings.items():
    print(f"  Cluster {cluster_id}: shape {np.array(embedding).shape}, mean={np.mean(embedding):.4f}, std={np.std(embedding):.4f}")

# Store in data_dict for later use
data_dict[name]['enhanced_dataset'] = enhanced_df
data_dict[name]['cluster_centers'] = cluster_centers
data_dict[name]['cluster_center_embeddings'] = cluster_center_embeddings

print(f"\n✅ Enhanced PR dataset with cluster information completed!")
print(f"🎯 New columns added:")
print(f"  1. 'cluster_id': Which cluster each sample belongs to")
print(f"  2. 'cluster_center_embedding': 768-dim embedding of the cluster center representative")

🔄 Creating enhanced PR dataset with cluster information...

📋 Processing 173 PR samples...
📊 Found 7 unique clusters (including noise): [np.int64(-1), np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
  Cluster 0: 113 points, center: [7.352, 1.543, 3.746]
  Cluster 1: 24 points, center: [4.780, 2.553, 1.370]
  Cluster 2: 5 points, center: [6.854, 1.659, 4.277]
  Cluster 3: 4 points, center: [6.844, 2.180, 3.611]
  Cluster 4: 3 points, center: [6.539, 1.776, 2.696]
  Cluster 5: 18 points, center: [3.936, 3.755, 2.758]

🔧 Handling 6 noise points...
  Noise point 1 assigned to nearest cluster 1 (distance: 1.369)
  Noise point 2 assigned to nearest cluster 1 (distance: 1.128)
  Noise point 3 assigned to nearest cluster 4 (distance: 1.190)
  Noise point 4 assigned to nearest cluster 4 (distance: 1.048)
  Noise point 5 assigned to nearest cluster 4 (distance: 0.603)
  Noise point 6 assigned to nearest cluster 0 (distance: 1.387)

📝 Creating enhanced dataset...
✅ E

In [13]:
# Export enhanced dataset to files
print("💾 Exporting enhanced PR dataset to files...\n")

# name = 'PR'
name = 'HER2'
enhanced_df = data_dict[name]['enhanced_dataset']

# Export to CSV (without the cluster_center_embedding column due to size)
csv_columns = [col for col in enhanced_df.columns if col != 'cluster_center_embedding']
csv_df = enhanced_df[csv_columns].copy()

csv_filename = 'enhanced_PR_dataset.csv'
csv_df.to_csv(csv_filename, index=False)
print(f"✅ Exported basic dataset to '{csv_filename}'")
print(f"   Columns: {list(csv_df.columns)}")
print(f"   Shape: {csv_df.shape}")

# Export complete dataset with embeddings to JSON
json_filename = 'enhanced_PR_dataset_with_embeddings.json'
enhanced_df.to_json(json_filename, orient='records', indent=2)
print(f"✅ Exported complete dataset with embeddings to '{json_filename}'")

# Export cluster center embeddings separately
cluster_centers_data = {
    'cluster_centers_umap': {str(k): v.tolist() for k, v in data_dict[name]['cluster_centers'].items()},
    'cluster_center_embeddings': {str(k): v.tolist() for k, v in data_dict[name]['cluster_center_embeddings'].items()}
}

cluster_filename = 'PR_cluster_centers_and_embeddings.json'
with open(cluster_filename, 'w') as f:
    json.dump(cluster_centers_data, f, indent=2)
print(f"✅ Exported cluster centers and embeddings to '{cluster_filename}'")

# Show detailed information about the enhanced dataset
print(f"\n📋 Enhanced Dataset Details:")
print(f"="*60)

print(f"\n🔍 Column Descriptions:")
print(f"  • sample_id: Original sample identifier")
print(f"  • pred: Model prediction (0/1)")
print(f"  • target: Ground truth label (0/1)")
print(f"  • predicts_correct: Whether prediction is correct (True/False)")
print(f"  • confusion_label: TP/TN/FP/FN category")
print(f"  • umap_x, umap_y, umap_z: 3D UMAP coordinates")
print(f"  • cluster_id: 🆕 Which cluster the sample belongs to")
print(f"  • cluster_center_embedding: 🆕 768-dim embedding of cluster center")

print(f"\n📊 Cluster Statistics:")
for cluster_id in sorted(data_dict[name]['cluster_center_embeddings'].keys()):
    cluster_mask = enhanced_df['cluster_id'] == cluster_id
    cluster_samples = enhanced_df[cluster_mask]

    accuracy = (cluster_samples['predicts_correct'] == True).sum() / len(cluster_samples) * 100
    tp_count = (cluster_samples['confusion_label'] == 'TP').sum()
    tn_count = (cluster_samples['confusion_label'] == 'TN').sum()
    fp_count = (cluster_samples['confusion_label'] == 'FP').sum()
    fn_count = (cluster_samples['confusion_label'] == 'FN').sum()

    print(f"  Cluster {cluster_id}:")
    print(f"    📊 Size: {len(cluster_samples)} samples")
    print(f"    ✅ Accuracy: {accuracy:.1f}%")
    print(f"    🟢 TP: {tp_count}, 🔵 TN: {tn_count}, 🔴 FP: {fp_count}, 🟡 FN: {fn_count}")

    embedding = data_dict[name]['cluster_center_embeddings'][cluster_id]
    print(f"    🧬 Center embedding: mean={np.mean(embedding):.4f}, std={np.std(embedding):.4f}")
    print()

# Handle noise points if any
noise_samples = enhanced_df[enhanced_df['cluster_id'].astype(str).str.contains('Noise')]
if len(noise_samples) > 0:
    print(f"🔧 Noise Points: {len(noise_samples)} samples")
    print(f"   These were assigned to nearest cluster centers for embedding purposes")

print(f"\n📁 Files Generated:")
print(f"  1. {csv_filename} - Basic dataset (CSV format)")
print(f"  2. {json_filename} - Complete dataset with embeddings (JSON)")
print(f"  3. {cluster_filename} - Cluster centers and embeddings (JSON)")

print(f"\n✅ Enhanced PR dataset export completed!")
print(f"🎯 You now have your original PR data with two new columns:")
print(f"   • cluster_id: Shows which of the 4 clusters each sample belongs to")
print(f"   • cluster_center_embedding: 768-dimensional embedding of that cluster's center")

💾 Exporting enhanced PR dataset to files...

✅ Exported basic dataset to 'enhanced_PR_dataset.csv'
   Columns: ['sample_id', 'pred', 'target', 'predicts_correct', 'confusion_label', 'umap_x', 'umap_y', 'umap_z', 'cluster_id']
   Shape: (173, 9)
✅ Exported complete dataset with embeddings to 'enhanced_PR_dataset_with_embeddings.json'
✅ Exported cluster centers and embeddings to 'PR_cluster_centers_and_embeddings.json'

📋 Enhanced Dataset Details:

🔍 Column Descriptions:
  • sample_id: Original sample identifier
  • pred: Model prediction (0/1)
  • target: Ground truth label (0/1)
  • predicts_correct: Whether prediction is correct (True/False)
  • confusion_label: TP/TN/FP/FN category
  • umap_x, umap_y, umap_z: 3D UMAP coordinates
  • cluster_id: 🆕 Which cluster the sample belongs to
  • cluster_center_embedding: 🆕 768-dim embedding of cluster center

📊 Cluster Statistics:
  Cluster 0:
    📊 Size: 113 samples
    ✅ Accuracy: 85.0%
    🟢 TP: 7, 🔵 TN: 89, 🔴 FP: 4, 🟡 FN: 13
    🧬 Center e

## 📊 Visualization 5: Spatial Distribution of Error Types

Analyze whether FP and FN samples have different spatial distributions

In [14]:
from scipy.spatial.distance import cdist

print("\n" + "="*70)
print("Spatial Distribution Analysis of Error Types")
print("="*70)

# for name in ['HER2', 'ER', 'PR']:
for name in ['HER2']:
    print(f"\n{name}:")
    print("-" * 60)

    umap_coords = data_dict[name]['umap']
    confusion_labels = data_dict[name]['confusion_labels']

    # Get coordinates for each category
    tp_coords = umap_coords[confusion_labels == 'TP']
    tn_coords = umap_coords[confusion_labels == 'TN']
    fp_coords = umap_coords[confusion_labels == 'FP']
    fn_coords = umap_coords[confusion_labels == 'FN']

    # Calculate average distances between error types
    if len(fp_coords) > 1:
        fp_distances = cdist(fp_coords, fp_coords, metric='euclidean')
        fp_avg_dist = fp_distances[np.triu_indices_from(fp_distances, k=1)].mean()
        print(f"  🔴 FP internal cohesion (avg distance): {fp_avg_dist:.3f}")

    if len(fn_coords) > 1:
        fn_distances = cdist(fn_coords, fn_coords, metric='euclidean')
        fn_avg_dist = fn_distances[np.triu_indices_from(fn_distances, k=1)].mean()
        print(f"  🟡 FN internal cohesion (avg distance): {fn_avg_dist:.3f}")

    # Distance between FP and FN
    if len(fp_coords) > 0 and len(fn_coords) > 0:
        fp_fn_distances = cdist(fp_coords, fn_coords, metric='euclidean')
        fp_fn_avg_dist = fp_fn_distances.mean()
        print(f"  🔴↔️🟡 FP to FN distance: {fp_fn_avg_dist:.3f}")

    # Distance from errors to correct samples
    correct_coords = np.vstack([tp_coords, tn_coords]) if len(tp_coords) > 0 and len(tn_coords) > 0 else (tp_coords if len(tp_coords) > 0 else tn_coords)

    if len(fp_coords) > 0 and len(correct_coords) > 0:
        fp_to_correct = cdist(fp_coords, correct_coords, metric='euclidean').min(axis=1)
        print(f"  🔴→✅ FP to nearest correct sample: {fp_to_correct.mean():.3f} (min: {fp_to_correct.min():.3f}, max: {fp_to_correct.max():.3f})")

    if len(fn_coords) > 0 and len(correct_coords) > 0:
        fn_to_correct = cdist(fn_coords, correct_coords, metric='euclidean').min(axis=1)
        print(f"  🟡→✅ FN to nearest correct sample: {fn_to_correct.mean():.3f} (min: {fn_to_correct.min():.3f}, max: {fn_to_correct.max():.3f})")

    # Interpretation
    if len(fp_coords) > 1 and len(fn_coords) > 1 and len(fp_coords) > 0 and len(fn_coords) > 0:
        print(f"\n  💡 Interpretation:")
        if fp_fn_avg_dist > max(fp_avg_dist, fn_avg_dist) * 1.5:
            print(f"     FP and FN form separate clusters (distance ratio: {fp_fn_avg_dist/max(fp_avg_dist, fn_avg_dist):.2f}x)")
        else:
            print(f"     FP and FN are spatially mixed (distance ratio: {fp_fn_avg_dist/max(fp_avg_dist, fn_avg_dist):.2f}x)")

print("\n" + "="*70)
print("✅ Spatial analysis completed")


Spatial Distribution Analysis of Error Types

HER2:
------------------------------------------------------------
  🔴 FP internal cohesion (avg distance): 2.852
  🟡 FN internal cohesion (avg distance): 2.636
  🔴↔️🟡 FP to FN distance: 2.639
  🔴→✅ FP to nearest correct sample: 0.242 (min: 0.123, max: 0.467)
  🟡→✅ FN to nearest correct sample: 0.244 (min: 0.087, max: 0.639)

  💡 Interpretation:
     FP and FN are spatially mixed (distance ratio: 0.93x)

✅ Spatial analysis completed


## 📊 Visualization 6: Confusion Matrix Heatmap Summary

Create a summary heatmap showing the distribution of four categories across biomarkers

In [15]:
# Prepare data for heatmap
confusion_summary = []

# for name in ['HER2', 'ER', 'PR']:
for name in ['PR']:
    confusion_labels = data_dict[name]['confusion_labels']
    total = len(confusion_labels)

    confusion_summary.append({
        'Biomarker': name,
        'TP': (confusion_labels == 'TP').sum(),
        'TN': (confusion_labels == 'TN').sum(),
        'FP': (confusion_labels == 'FP').sum(),
        'FN': (confusion_labels == 'FN').sum()
    })

summary_df = pd.DataFrame(confusion_summary)
print("\nConfusion Matrix Summary:")
print(summary_df.to_string(index=False))

# Create stacked bar chart
fig = go.Figure()

categories = ['TP', 'TN', 'FP', 'FN']
colors_map = {
    'TP': '#27AE60',
    'TN': '#3498DB',
    'FP': '#E74C3C',
    'FN': '#F39C12'
}
names_map = {
    'TP': '🟢 True Positive',
    'TN': '🔵 True Negative',
    'FP': '🔴 False Positive',
    'FN': '🟡 False Negative'
}

for category in categories:
    fig.add_trace(go.Bar(
        name=names_map[category],
        x=summary_df['Biomarker'],
        y=summary_df[category],
        marker_color=colors_map[category],
        text=summary_df[category],
        textposition='inside',
        textfont=dict(color='white', size=14, family='Arial Black')
    ))

fig.update_layout(
    title='Confusion Matrix Distribution Across Three Biomarkers',
    xaxis_title='Biomarker',
    yaxis_title='Sample Count',
    barmode='stack',
    height=600,
    width=1000,
    showlegend=True,
    legend=dict(
        yanchor="middle",
        y=0.5,
        xanchor="left",
        x=1.02,
        orientation="v",
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="rgba(0, 0, 0, 0.2)",
        borderwidth=1
    )
)

fig.show()

# Calculate and display percentages
print("\n" + "="*60)
print("Percentage Distribution:")
print("="*60)
for _, row in summary_df.iterrows():
    total = row['TP'] + row['TN'] + row['FP'] + row['FN']
    print(f"\n{row['Biomarker']}:")
    print(f"  🟢 TP: {row['TP']:3d} ({row['TP']/total*100:5.1f}%)")
    print(f"  🔵 TN: {row['TN']:3d} ({row['TN']/total*100:5.1f}%)")
    print(f"  🔴 FP: {row['FP']:3d} ({row['FP']/total*100:5.1f}%)")
    print(f"  🟡 FN: {row['FN']:3d} ({row['FN']/total*100:5.1f}%)")
    print(f"  ✅ Accuracy: {(row['TP']+row['TN'])/total*100:.1f}%")

KeyError: 'PR'

## 📊 Visualization 7: 3D Multi-Dimensional Comparison Analysis

Compare different visualization perspectives in a 4-panel 3D layout:
- **Panel 1**: By Prediction Correctness (Predicts: True/False) - 3D
- **Panel 2**: By Prediction Results (preds: 0/1) - 3D
- **Panel 3**: By Ground Truth (targets: 0/1) - 3D
- **Panel 4**: By Confusion Matrix (TP/TN/FP/FN) - 3D

In [ ]:
# Select a biomarker for multi-dimensional comparison (can be changed to 'ER' or 'PR')
selected_marker = 'PR'

# Create 2x2 subplot layout for 3D visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        f'{selected_marker} - By Prediction Correctness',
        f'{selected_marker} - By Prediction Results',
        f'{selected_marker} - By Ground Truth',
        f'{selected_marker} - By Confusion Matrix'
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.15,
    specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}],
           [{"type": "scatter3d"}, {"type": "scatter3d"}]]  # All 3D plots
)

umap_coords = data_dict[selected_marker]['umap']
sample_ids = data_dict[selected_marker]['sample_ids']
preds = data_dict[selected_marker]['preds']
targets = data_dict[selected_marker]['targets']
predicts = data_dict[selected_marker]['predicts']
confusion_labels = data_dict[selected_marker]['confusion_labels']

# Panel 1: By Prediction Correctness (Predicts) - 3D
print(f"Creating 3D multi-dimensional comparison for {selected_marker}...\n")

for correct_val, label, color in [(True, 'Correct', '#2ECC71'), (False, 'Wrong', '#E74C3C')]:
    mask = predicts == correct_val
    fig.add_trace(
        go.Scatter3d(  # Changed to 3D
            x=umap_coords[mask, 0],
            y=umap_coords[mask, 1],
            z=umap_coords[mask, 2],  # Added z-coordinate
            mode='markers',
            name=f'Predicts: {label}',
            marker=dict(
                size=5 if not correct_val else 5,
                color=color,
                opacity=1.0 if not correct_val else 0.6,
                symbol='x' if not correct_val else 'circle',
                line=dict(width=2 if not correct_val else 1, color='white')
            ),
            text=[f"Sample: {sample_ids[i]}<br>Predicts: {label}<br>Pred: {preds[i]} | True: {targets[i]}"
                  for i in np.where(mask)[0]],
            hovertemplate='<b>%{text}</b><br>UMAP1: %{x:.2f}<br>UMAP2: %{y:.2f}<br>UMAP3: %{z:.2f}<extra></extra>',
            showlegend=True,
            legendgroup=f'predicts_{label}',
            legendgrouptitle_text="Prediction Correctness"
        ),
        row=1, col=1
    )

# Panel 2: By Prediction Results (preds) - 3D
for pred_val, label, color in [(1, 'Positive', '#E67E22'), (0, 'Negative', '#8E44AD')]:
    mask = preds == pred_val
    fig.add_trace(
        go.Scatter3d(  # Changed to 3D
            x=umap_coords[mask, 0],
            y=umap_coords[mask, 1],
            z=umap_coords[mask, 2],  # Added z-coordinate
            mode='markers',
            name=f'Pred: {label} ({pred_val})',
            marker=dict(
                size=5,
                color=color,
                opacity=0.7,
                line=dict(width=1, color='white')
            ),
            text=[f"Sample: {sample_ids[i]}<br>Predicted: {label} ({pred_val})<br>True: {targets[i]}"
                  for i in np.where(mask)[0]],
            hovertemplate='<b>%{text}</b><br>UMAP1: %{x:.2f}<br>UMAP2: %{y:.2f}<br>UMAP3: %{z:.2f}<extra></extra>',
            showlegend=True,
            legendgroup=f'preds_{pred_val}',
            legendgrouptitle_text="Prediction Results"
        ),
        row=1, col=2
    )

# Panel 3: By Ground Truth (targets) - 3D
for target_val, label, color in [(1, 'Positive', '#16A085'), (0, 'Negative', '#34495E')]:
    mask = targets == target_val
    fig.add_trace(
        go.Scatter3d(  # Changed to 3D
            x=umap_coords[mask, 0],
            y=umap_coords[mask, 1],
            z=umap_coords[mask, 2],  # Added z-coordinate
            mode='markers',
            name=f'True: {label} ({target_val})',
            marker=dict(
                size=5,
                color=color,
                opacity=0.7,
                line=dict(width=1, color='white')
            ),
            text=[f"Sample: {sample_ids[i]}<br>Ground Truth: {label} ({target_val})<br>Pred: {preds[i]}"
                  for i in np.where(mask)[0]],
            hovertemplate='<b>%{text}</b><br>UMAP1: %{x:.2f}<br>UMAP2: %{y:.2f}<br>UMAP3: %{z:.2f}<extra></extra>',
            showlegend=True,
            legendgroup=f'targets_{target_val}',
            legendgrouptitle_text="Ground Truth"
        ),
        row=2, col=1
    )

# Panel 4: By Confusion Matrix - 3D
for category in category_order:
    mask = confusion_labels == category
    config = confusion_config[category]

    if mask.sum() > 0:
        fig.add_trace(
            go.Scatter3d(  # Changed to 3D
                x=umap_coords[mask, 0],
                y=umap_coords[mask, 1],
                z=umap_coords[mask, 2],  # Added z-coordinate
                mode='markers',
                name=config['name'],
                marker=dict(
                    size=config['size'],
                    color=config['color'],
                    symbol=config['symbol'] if config['symbol'] in ['circle', 'square', 'diamond'] else 'circle',
                    opacity=config['opacity'],
                    line=dict(width=1.5, color='white')
                ),
                text=[f"Sample: {sample_ids[i]}<br>{config['name']}<br>Pred: {preds[i]} | True: {targets[i]}"
                      for i in np.where(mask)[0]],
                hovertemplate='<b>%{text}</b><br>UMAP1: %{x:.2f}<br>UMAP2: %{y:.2f}<br>UMAP3: %{z:.2f}<extra></extra>',
                showlegend=True,
                legendgroup=f'confusion_{category}',
                legendgrouptitle_text="Confusion Matrix"
            ),
            row=2, col=2
        )

# Update layout for 3D visualizations
fig.update_layout(
    title_text=f'{selected_marker} - 3D Multi-Dimensional Comparison Analysis<br><sub>Four different 3D perspectives on the same data</sub>',
    height=1200,
    width=1600,
    showlegend=True,
    legend=dict(
        yanchor="middle",
        y=0.5,
        xanchor="left",
        x=1.01,
        orientation="v",
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="rgba(0, 0, 0, 0.2)",
        borderwidth=1,
        tracegroupgap=15
    )
)

# Update 3D scene properties for all subplots
fig.update_layout(
    scene=dict(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        zaxis_title='UMAP 3',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    ),
    scene2=dict(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        zaxis_title='UMAP 3',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    ),
    scene3=dict(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        zaxis_title='UMAP 3',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    ),
    scene4=dict(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        zaxis_title='UMAP 3',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    )
)

fig.show()

# Print comparison statistics
print("\n" + "="*70)
print(f"3D Multi-Dimensional Analysis Summary for {selected_marker}")
print("="*70)

print("\n📊 Dimension 1 - Prediction Correctness:")
correct_count = predicts.sum()
wrong_count = len(predicts) - correct_count
print(f"  ✅ Correct: {correct_count} ({correct_count/len(predicts)*100:.1f}%)")
print(f"  ❌ Wrong:   {wrong_count} ({wrong_count/len(predicts)*100:.1f}%)")

print("\n📊 Dimension 2 - Prediction Results:")
pos_pred = (preds == 1).sum()
neg_pred = (preds == 0).sum()
print(f"  Predicted Positive: {pos_pred} ({pos_pred/len(preds)*100:.1f}%)")
print(f"  Predicted Negative: {neg_pred} ({neg_pred/len(preds)*100:.1f}%)")

print("\n📊 Dimension 3 - Ground Truth:")
true_pos = (targets == 1).sum()
true_neg = (targets == 0).sum()
print(f"  True Positive: {true_pos} ({true_pos/len(targets)*100:.1f}%)")
print(f"  True Negative: {true_neg} ({true_neg/len(targets)*100:.1f}%)")

print("\n📊 Dimension 4 - Confusion Matrix:")
for category in category_order:
    count = (confusion_labels == category).sum()
    config = confusion_config[category]
    print(f"  {config['name']}: {count} ({count/len(confusion_labels)*100:.1f}%)")

print("\n💡 Key Observations in 3D:")
print("  • Panel 1 shows 3D prediction accuracy distribution")
print("  • Panel 2 shows 3D model prediction bias (positive vs negative)")
print("  • Panel 3 shows 3D actual data distribution")
print("  • Panel 4 shows 3D detailed error types")
print("  • 3D visualization reveals hidden clustering patterns")
print("  • Z-axis provides additional separation for complex cases")

print("\n✅ 3D multi-dimensional comparison completed!")

## 📊 Visualization 8: 3D PR Biomarker Analysis Views

Compare different 3D visualization perspectives for the PR biomarker

In [ ]:
# Create a function to generate 3D comparison views (adjusted for PR only)
def create_comparison_view(view_type='confusion'):
    """
    Create 3D visualization for PR biomarker only
    view_type: 'confusion', 'predicts', 'preds', or 'targets'
    """
    fig = go.Figure()  # Single 3D plot

    # Only process PR data
    name = 'PR'
    umap_coords = data_dict[name]['umap']
    sample_ids = data_dict[name]['sample_ids']
    preds = data_dict[name]['preds']
    targets = data_dict[name]['targets']
    predicts = data_dict[name]['predicts']
    confusion_labels = data_dict[name]['confusion_labels']

    if view_type == 'confusion':
        # Confusion matrix view - 3D
        for category in category_order:
            mask = confusion_labels == category
            config = confusion_config[category]

            if mask.sum() > 0:
                fig.add_trace(
                    go.Scatter3d(  # Changed to 3D
                        x=umap_coords[mask, 0],
                        y=umap_coords[mask, 1],
                        z=umap_coords[mask, 2],  # Added z-coordinate
                        mode='markers',
                        name=config['name'],
                        marker=dict(
                            size=config['size'],
                            color=config['color'],
                            symbol='circle',  # Simplified for 3D
                            opacity=config['opacity'],
                            line=dict(width=1.5, color='white')
                        ),
                        text=[f"Sample: {sample_ids[i]}<br>{name}<br>{config['name']}"
                              for i in np.where(mask)[0]],
                        hovertemplate='<b>%{text}</b><br>UMAP1: %{x:.2f}<br>UMAP2: %{y:.2f}<br>UMAP3: %{z:.2f}<extra></extra>',
                        showlegend=True,
                        legendgroup=category
                    )
                )
        title = f'{name} - 3D Confusion Matrix Analysis'

    elif view_type == 'predicts':
        # Prediction correctness view - 3D
        for correct_val, label, color in [(True, 'Correct', '#2ECC71'), (False, 'Wrong', '#E74C3C')]:
            mask = predicts == correct_val
            fig.add_trace(
                go.Scatter3d(  # Changed to 3D
                    x=umap_coords[mask, 0],
                    y=umap_coords[mask, 1],
                    z=umap_coords[mask, 2],  # Added z-coordinate
                    mode='markers',
                    name=label,
                    marker=dict(
                        size=5,  # Changed from 10/8 to 5 for consistency
                        color=color,
                        opacity=1.0 if not correct_val else 0.6,
                        symbol='circle',  # Simplified for 3D
                        line=dict(width=2 if not correct_val else 1, color='white')
                    ),
                    text=[f"Sample: {sample_ids[i]}<br>{name}<br>Predicts: {label}"
                          for i in np.where(mask)[0]],
                    hovertemplate='<b>%{text}</b><br>UMAP1: %{x:.2f}<br>UMAP2: %{y:.2f}<br>UMAP3: %{z:.2f}<extra></extra>',
                    showlegend=True,
                    legendgroup=label
                )
            )
        title = f'{name} - 3D Prediction Correctness'

    elif view_type == 'preds':
        # Prediction results view - 3D
        for pred_val, label, color in [(1, 'Positive', '#E67E22'), (0, 'Negative', '#8E44AD')]:
            mask = preds == pred_val
            fig.add_trace(
                go.Scatter3d(  # Changed to 3D
                    x=umap_coords[mask, 0],
                    y=umap_coords[mask, 1],
                    z=umap_coords[mask, 2],  # Added z-coordinate
                    mode='markers',
                    name=f'Pred: {label}',
                    marker=dict(
                        size=5,  # Changed from 8 to 5 for consistency
                        color=color,
                        opacity=0.7,
                        line=dict(width=1, color='white')
                    ),
                    text=[f"Sample: {sample_ids[i]}<br>{name}<br>Predicted: {label}"
                          for i in np.where(mask)[0]],
                    hovertemplate='<b>%{text}</b><br>UMAP1: %{x:.2f}<br>UMAP2: %{y:.2f}<br>UMAP3: %{z:.2f}<extra></extra>',
                    showlegend=True,
                    legendgroup=label
                )
            )
        title = f'{name} - 3D Prediction Results'

    else:  # targets
        # Ground truth view - 3D
        for target_val, label, color in [(1, 'Positive', '#16A085'), (0, 'Negative', '#34495E')]:
            mask = targets == target_val
            fig.add_trace(
                go.Scatter3d(  # Changed to 3D
                    x=umap_coords[mask, 0],
                    y=umap_coords[mask, 1],
                    z=umap_coords[mask, 2],  # Added z-coordinate
                    mode='markers',
                    name=f'True: {label}',
                    marker=dict(
                        size=5,  # Changed from 8 to 5 for consistency
                        color=color,
                        opacity=0.7,
                        line=dict(width=1, color='white')
                    ),
                    text=[f"Sample: {sample_ids[i]}<br>{name}<br>Ground Truth: {label}"
                          for i in np.where(mask)[0]],
                    hovertemplate='<b>%{text}</b><br>UMAP1: %{x:.2f}<br>UMAP2: %{y:.2f}<br>UMAP3: %{z:.2f}<extra></extra>',
                    showlegend=True,
                    legendgroup=label
                )
            )
        title = f'{name} - 3D Ground Truth'

    # Update layout for 3D single plot
    fig.update_layout(
        title_text=title,
        scene=dict(
            xaxis_title='UMAP 1',
            yaxis_title='UMAP 2',
            zaxis_title='UMAP 3',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
            aspectmode='cube'
        ),
        height=800,
        width=1000,
        showlegend=True,
        legend=dict(
            yanchor="middle",
            y=0.5,
            xanchor="left",
            x=1.02,
            orientation="v",
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="rgba(0, 0, 0, 0.2)",
            borderwidth=1
        )
    )

    return fig

# Create all four 3D comparison views for PR only
print("Creating 3D visualization views for PR biomarker...\n")

print("1️⃣ By Prediction Correctness (Predicts) - 3D:")
fig1 = create_comparison_view('predicts')
fig1.show()

print("\n2️⃣ By Prediction Results (preds) - 3D:")
fig2 = create_comparison_view('preds')
fig2.show()

print("\n3️⃣ By Ground Truth (targets) - 3D:")
fig3 = create_comparison_view('targets')
fig3.show()

print("\n4️⃣ By Confusion Matrix (TP/TN/FP/FN) - 3D:")
fig4 = create_comparison_view('confusion')
fig4.show()

print("\n✅ All PR 3D visualization views generated!")
print("🔄 3D Interactive Features Available:")
print("   • Click and drag to rotate each 3D plot")
print("   • Use mouse wheel to zoom in/out")
print("   • Hold Shift and drag to pan")
print("   • Double-click to reset view")
print("   • Click legend items to show/hide categories")

## 📝 Summary Report

Key findings from confusion matrix analysis:

In [ ]:

print("\n" + "="*70)
print("📊 CONFUSION MATRIX ANALYSIS - SUMMARY REPORT (PR ONLY)")
print("="*70)

print("\n1️⃣ Overall Performance Metrics:")
print("-" * 60)

# Only analyze PR data
name = 'PR'
confusion_labels = data_dict[name]['confusion_labels']

tp = (confusion_labels == 'TP').sum()
tn = (confusion_labels == 'TN').sum()
fp = (confusion_labels == 'FP').sum()
fn = (confusion_labels == 'FN').sum()

total = len(confusion_labels)
accuracy = (tp + tn) / total
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\n{name}:")
print(f"  Accuracy:    {accuracy*100:6.2f}%")
print(f"  Precision:   {precision*100:6.2f}%  (TP / (TP + FP))")
print(f"  Recall:      {recall*100:6.2f}%  (TP / (TP + FN))")
print(f"  Specificity: {specificity*100:6.2f}%  (TN / (TN + FP))")
print(f"  F1-Score:    {f1*100:6.2f}%")

print(f"\nDetailed counts:")
print(f"  🟢 True Positives (TP):  {tp:3d}")
print(f"  🔵 True Negatives (TN):  {tn:3d}")
print(f"  🔴 False Positives (FP): {fp:3d}")
print(f"  🟡 False Negatives (FN): {fn:3d}")
print(f"  📊 Total samples:        {total:3d}")

print("\n2️⃣ Error Pattern Analysis:")
print("-" * 60)

if fp + fn > 0:
    print(f"\n{name}:")
    print(f"  Total Errors: {fp + fn}")
    print(f"  🔴 False Positives: {fp} ({fp/(fp+fn)*100:.1f}% of errors)")
    print(f"  🟡 False Negatives: {fn} ({fn/(fp+fn)*100:.1f}% of errors)")

    if fp > fn * 1.5:
        print(f"  ⚠️  Model tends to over-predict (more FP)")
    elif fn > fp * 1.5:
        print(f"  ⚠️  Model tends to under-predict (more FN)")
    else:
        print(f"  ✅ Error types are balanced")
else:
    print(f"\n{name}: No prediction errors found!")



## 💡 Next Steps

### ✅ **3D Conversion Complete**
All visualizations have been successfully converted to interactive 3D plots using UMAP 3D coordinates.

### Possible Further Analyses:

1. **Threshold Adjustment**: Try different classification thresholds to balance FP/FN in 3D space
2. **3D Feature Analysis**: Compare 3D embedding features between FP and FN samples
3. **Clinical Impact**: Assess which error type (FP vs FN) has more serious consequences using 3D clustering
4. **Ensemble Learning**: Combine predictions from three biomarkers in 3D space
5. **3D Error Case Study**: Deep dive into specific misclassified samples using 3D coordinates
6. **Advanced 3D Analysis**: Use 3D clustering algorithms (DBSCAN, K-means) on 3D embeddings
7. **4D Visualization**: Add time or additional clinical parameters as color/size encoding

### 🔄 **3D Interactive Features Available:**
- **Rotation**: Click and drag to rotate any 3D plot from different angles
- **Zoom**: Use mouse wheel to zoom in/out for detailed inspection
- **Pan**: Hold Shift and drag to pan across the 3D space
- **Reset**: Double-click to reset view to original position
- **Legend**: Click legend items to show/hide specific categories
- **Hover**: Detailed sample information with 3D coordinates (x, y, z)

### 📊 **3D Visualization Benefits:**
- **Enhanced Separation**: Z-axis provides additional dimension for separating confusion matrix categories
- **Hidden Patterns**: 3D reveals clustering patterns not visible in 2D projections
- **Better Understanding**: Interactive rotation helps understand spatial relationships
- **Improved Analysis**: 3D coordinates enable more sophisticated clustering and classification analysis